https://spaces.ac.cn/archives/8069

# numpy版本

In [2]:
import pickle
import numpy as np

class Whitening(object):
    def __init__(self, reps):
        self.question_reps = reps
            
    def save_kernel_bias(self, path):
        whiten = {
            'kernel': self.kernel,
            'bias': self.bias
        }
        with open(path, 'wb') as f:
            pickle.dump(whiten, f)
        return
    
    def load_kernel_bias(self, path):
        with open(path, 'rb') as f:
            whiten = pickle.load(f)
        return whiten['kernel'], whiten['bias']
        

    def compute_kernel_bias(self):
        """计算kernel和bias
        最后的变换：y = (x + bias).dot(kernel)
        """
        mu = self.question_reps.mean(axis=0, keepdims=True)
        cov = np.cov(self.question_reps.T)
        u, s, vh = np.linalg.svd(cov)
        W = np.dot(u, np.diag(1/np.sqrt(s)))
        return W, -mu

    def transform_and_normalize(self, vecs):
        """应用变换，然后标准化
        """
        if not (self.kernel is None or self.bias is None):
            vecs = (vecs + self.bias).dot(self.kernel)
        return vecs / (vecs**2).sum(axis=1, keepdims=True)**0.5

    def whitening(self):
        self.kernel, self.bias = self.compute_kernel_bias()
        whitened_reps = []
        for i in range(self.question_reps.shape[0]):
            whitened_reps.append(self.transform_and_normalize(self.question_reps[i, :]).squeeze(0))
        return np.array(whitened_reps).astype("float32")

In [3]:
data = np.random.rand(5, 2)
print(data)

compute = Whitening(data)
compute.whitening()

[[0.44030528 0.92999532]
 [0.69068517 0.59581432]
 [0.71507307 0.43196052]
 [0.6306014  0.61970793]
 [0.44024523 0.17010015]]


array([[ 0.8158315 , -0.57828975],
       [ 0.18700892,  0.9823582 ],
       [-0.41579348,  0.90945905],
       [ 0.5567029 ,  0.83071166],
       [-0.7591021 , -0.6509716 ]], dtype=float32)

# torch版本

In [8]:
import torch

def whitening_torch(embed):
    # For torch < 1.10
    mu = torch.mean(embed, dim=0, keepdim=True)
    # cov = torch.mm((embeddings - mu).t(), (embeddings - mu))
    # For torch >= 1.10
    cov = torch.cov(embed.t()) # 计算协方差矩阵。描述的是一组随机变量两两之间的相关性大小
    
    u, s, vt = torch.svd(cov)    # 奇异值分解，输出 左奇异向量矩阵，奇异值向量，右奇异向量矩阵
    W = torch.mm(u, torch.diag(1/torch.sqrt(s)))
    embed = torch.mm(embed - mu, W)
    return embed

torch.nn.functional.normalize(whitening_torch(torch.from_numpy(data)))

tensor([[ 0.8158, -0.5783],
        [ 0.1870,  0.9824],
        [-0.4158,  0.9095],
        [ 0.5567,  0.8307],
        [-0.7591, -0.6510]], dtype=torch.float64)

# 查看转换后距离变化

距离的大小变化了，不仅数值变化，连相对大小关系、顺序都变化了。

In [9]:
c = torch.from_numpy(data)

In [10]:
torch.nn.functional.cosine_similarity(c.unsqueeze(dim=1), c.unsqueeze(dim=0), dim=2)   # [5, 1, 2] [1, 5, 2]

tensor([[1.0000, 0.9144, 0.8336, 0.9387, 0.7249],
        [0.9144, 1.0000, 0.9859, 0.9979, 0.9417],
        [0.8336, 0.9859, 1.0000, 0.9729, 0.9848],
        [0.9387, 0.9979, 0.9729, 1.0000, 0.9179],
        [0.7249, 0.9417, 0.9848, 0.9179, 1.0000]], dtype=torch.float64)

In [11]:
c2 = torch.nn.functional.normalize(whitening_torch(c))

In [12]:
torch.nn.functional.cosine_similarity(c2.unsqueeze(dim=1), c2.unsqueeze(dim=0), dim=2)

tensor([[ 1.0000, -0.4155, -0.8651, -0.0262, -0.2428],
        [-0.4155,  1.0000,  0.8157,  0.9202, -0.7814],
        [-0.8651,  0.8157,  1.0000,  0.5240, -0.2764],
        [-0.0262,  0.9202,  0.5240,  1.0000, -0.9634],
        [-0.2428, -0.7814, -0.2764, -0.9634,  1.0000]], dtype=torch.float64)